In [1]:
import pandas as pd

In [2]:
%run Kapacity_skolskych_zarizeni.ipynb

## DS, MŠ + PSZŠSpeciální

### MŠ + PSZŠSpeciální (kapacita) TAB

In [3]:
materske_skoly_zarizeni = ["Mateřská škola", "Mateřská škola (lesní mateřská škola)", "Mateřská škola - Przedszkole", "Mateřská škola s p.j.v.", "Přípravný stupeň základní školy speciální"]
materske_skoly = skoly[skoly['ZarizeniTyp'].isin(materske_skoly_zarizeni)]
materske_skoly = materske_skoly.astype({'Kapacita':'int'})
# materske_skoly.info()
# materske_skoly['Kapacita'].sum()
materske_skoly

,IZO,ZarizeniTyp,Kapacita,KapacitaJednotka
0,049625918,Mateřská škola,54,dítě
2,107500884,Mateřská škola,70,dítě
4,110034384,Mateřská škola,6,dítě
6,060437171,Mateřská škola,130,dítě
8,110035585,Mateřská škola,18,dítě
...,...,...,...,...
27171,181145014,Mateřská škola (lesní mateřská škola),15,dítě
27177,181145499,Mateřská škola (lesní mateřská škola),15,dítě
27181,181145871,Mateřská škola,44,dítě
27187,181146878,Mateřská škola (lesní mateřská škola),16,dítě


**Je potřeba provést kontrolu, zda jedno MŠ(školské) zařízení nefiguruje ve vícero krajích. Což se bohužel potvrdilo.** Přesné kapacity každého zařízení se nedají dohledat na internetu.\
Nejčastěji se však jedná o rozmělnění v rámci Hlavní město Praha a Středočeský Kraj. Do počtu zařízení pak vede Praha, proto výsledky v tomto konktrétním případě vždy přiřadíme právě Praze.\
V případě jiné kombinaci sdílení krajů pro IZO zařízení vždy přiřadíme kapacity jinému kraji, než je Hlavní město Praha a Středočeský Kraj. Případně přiřadíme první vyhledaný kraj. Jedná se o desítky až nižší stovky míst zkreslení dat v rámci regionů se . \
Prozatím se neobjevil případ, kdy by pro jedno IZO zařízení bylo přiřazování více krajů než dva. **Manuální kontrola (zakomentováno níže).**

In [4]:
ms_duplicates = zarizeni_misto[zarizeni_misto['IZO'].isin(materske_skoly['IZO'])].groupby('IZO')['Kraj'].nunique()
shared_ms_kraje = ms_duplicates[ms_duplicates > 1].index
# shared_ms_kraje = ms_duplicates[ms_duplicates > 2].index # pro kontrolu, zda zařízení nefigurující ve vícero, než 2 krajích

# shared_ms_kraje
zarizeni_misto[zarizeni_misto['IZO'].isin(shared_ms_kraje)] # dataframe, který zobrazí zařízení sdílená s vícero kraji 
# materske_skoly[materske_skoly['IZO'].isin(shared_ms_kraje)] # dataframe, který nám zobrazí kapacity zařízení, která náleží vícero krajům
# materske_skoly[materske_skoly['IZO'].isin(shared_ms_kraje)]['Kapacita'].sum() # přesný výpočet kapacity celkem, která se dělí mezi vícero krajů (nejvíc jí patří do Prahy)
materske_skoly[materske_skoly['IZO'] == '181142180'] # jediné zařízení v rámci mš, které vůbec nemá vyplněné místo

,IZO,ZarizeniTyp,Kapacita,KapacitaJednotka
27088,181142180,Mateřská škola,16,dítě


spojit tabulky, opodmínkovat to, že ty, keré mají izo in shared_ms_kraje

In [5]:
def assignKraj(row):
    MS_IZO = row['IZO']
    
    if MS_IZO not in shared_ms_kraje:
        matching_row = zarizeni_misto[zarizeni_misto['IZO'] == MS_IZO]
        if not matching_row.empty:
            return matching_row['Kraj'].values[0] 
    elif MS_IZO in shared_ms_kraje:
        matching_row = zarizeni_misto[zarizeni_misto['IZO'] == MS_IZO]
        kraje = matching_row['Kraj'].unique()

        # Podmínka pro 'Hlavní město Praha' a 'Středočeský kraj'
        if 'Hlavní město Praha' in kraje and 'Středočeský Kraj' in kraje:
            return 'Hlavní město Praha'

        # Podmínka pro 'Hlavní město Praha' a jakýkoliv jiný kraj
        elif 'Hlavní město Praha' in kraje and len(kraje) > 1:
            other_kraje = [k for k in kraje if k != 'Hlavní město Praha']
            return other_kraje[0]  # Vrací první jiný kraj než 'Hlavní město Praha'

        # Podmínka pro 'Středočeský kraj' a jakýkoliv jiný kraj
        elif 'Středočeský kraj' in kraje and len(kraje) > 1:
            other_kraje = [k for k in kraje if k != 'Středočeský kraj']
            return other_kraje[0]  # Vrací první jiný kraj než 'Středočeský kraj'
                    
        else:
            return matching_row['Kraj'].values[0] # Vrať první kraj, pokud podmínky neplatí.
            
    else:
        return None
        
materske_skoly['Kraj'] = materske_skoly.apply(
    assignKraj, axis=1) 

In [6]:
# materske_skoly[materske_skoly['Kraj'].isnull()] #IZO 181142180 -> mš vůbec nemá zarizeni_misto

In [7]:
materske_skoly.loc[materske_skoly['IZO'] == '181142180', 'Kraj'] = 'Jihočeský kraj'
materske_skoly.info()
materske_skoly['Kraj'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Index: 5487 entries, 0 to 27188
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   IZO               5487 non-null   object
 1   ZarizeniTyp       5487 non-null   object
 2   Kapacita          5487 non-null   int64 
 3   KapacitaJednotka  5482 non-null   object
 4   Kraj              5487 non-null   object
dtypes: int64(1), object(4)
memory usage: 257.2+ KB


Kraj
Středočeský kraj        840
Jihomoravský kraj       694
Moravskoslezský kraj    480
Hlavní město Praha      448
Olomoucký kraj          391
Ústecký kraj            368
Jihočeský kraj          342
Zlínský kraj            333
Pardubický kraj         328
Královéhradecký kraj    323
Kraj Vysočina           292
Plzeňský kraj           281
Liberecký kraj          238
Karlovarský kraj        129
Name: count, dtype: int64

### DětskéSkupinky (realtime kapacita) TAB

In [13]:
%run ../data/web_scrap/web_scrap_DS
DS_tabulka = DS_tab

In [14]:
DS_tabulka = DS_tabulka.astype({'Kapacita':'int'})
# DS_tabulka.info()
DS_tabulka

,ZarizeniTyp,Kapacita,Kraj
0,Dětská skupinka,4633,Hlavní město Praha
1,Dětská skupinka,1311,Jihočeský kraj
2,Dětská skupinka,4075,Jihomoravský kraj
3,Dětská skupinka,262,Karlovarský kraj
4,Dětská skupinka,1147,Královéhradecký kraj
5,Dětská skupinka,794,Liberecký kraj
6,Dětská skupinka,2200,Moravskoslezský kraj
7,Dětská skupinka,1386,Olomoucký kraj
8,Dětská skupinka,1116,Pardubický kraj
9,Dětská skupinka,1077,Plzeňský kraj


### **Předškolní vzdělávání a péče TAB (kapacita a realtime kapacita)**

In [15]:
predskolniVP = pd.concat([materske_skoly, DS_tabulka], ignore_index=True, sort=False)
predskolniVP.info()
predskolniVP

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5501 entries, 0 to 5500
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   IZO               5487 non-null   object
 1   ZarizeniTyp       5501 non-null   object
 2   Kapacita          5501 non-null   int64 
 3   KapacitaJednotka  5482 non-null   object
 4   Kraj              5501 non-null   object
dtypes: int64(1), object(4)
memory usage: 215.0+ KB


,IZO,ZarizeniTyp,Kapacita,KapacitaJednotka,Kraj
0,049625918,Mateřská škola,54,dítě,Hlavní město Praha
1,107500884,Mateřská škola,70,dítě,Hlavní město Praha
2,110034384,Mateřská škola,6,dítě,Hlavní město Praha
3,060437171,Mateřská škola,130,dítě,Hlavní město Praha
4,110035585,Mateřská škola,18,dítě,Hlavní město Praha
...,...,...,...,...,...
5496,NaN,Dětská skupinka,1077,NaN,Plzeňský kraj
5497,NaN,Dětská skupinka,4106,NaN,Středočeský kraj
5498,NaN,Dětská skupinka,1595,NaN,Ústecký kraj
5499,NaN,Dětská skupinka,579,NaN,Kraj Vysočina


In [16]:
predskolniVP.to_csv(r'..\output_files\predskolniVP.csv', mode='w')